In [1]:
import os
import subprocess
import time
import sys
import threading
import socket
from urllib.request import Request, urlopen
import re
# Function to install packages
def install_packages():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "streamlit", "pyngrok", "pyjwt", "watchdog"])
print("Installing required packages...")
install_packages()
# Import after installation
from pyngrok import ngrok
# --- Create Streamlit Config for Dark Theme ---
os.makedirs(".streamlit", exist_ok=True)
config_toml = """
[theme]
base="dark"
primaryColor="#4F8BF9"
backgroundColor="#0E1117"
secondaryBackgroundColor="#262730"
textColor="#FAFAFA"
font="sans serif"
[server]
headless = true
"""
with open(".streamlit/config.toml", "w") as f:
    f.write(config_toml)
print("Applied Dark Theme configuration.")

Installing required packages...
Applied Dark Theme configuration.


In [ ]:
# --- Create the Streamlit App File ---
app_code = """
import streamlit as st
import jwt
import datetime
import sqlite3
import re

# ---------------- CONFIG ----------------
SECRET_KEY = "super_secret_key"
ALGORITHM = "HS256"
DB_NAME = "users.db"

# ---------------- DATABASE ----------------
conn = sqlite3.connect(DB_NAME, check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    username TEXT,
    email TEXT PRIMARY KEY,
    password TEXT,
    security_question TEXT,
    security_answer TEXT
)
''')
conn.commit()

# ---------------- JWT ----------------
def create_token(data):
    data["exp"] = datetime.datetime.utcnow() + datetime.timedelta(hours=1)
    return jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)

# ---------------- VALIDATIONS ----------------
def valid_email(email):
    pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
    return re.match(pattern, email)

def valid_password(password):
    return password.isalnum()

# ---------------- SESSION ----------------
if "page" not in st.session_state:
    st.session_state.page = "login"

if "token" not in st.session_state:
    st.session_state.token = None

# ---------------- SIGNUP ----------------
def signup():
    st.title("Signup Page")

    questions = [
        "What is your pet name?",
        "What is your mother’s maiden name?",
        "What is your favorite teacher?"
    ]

    with st.form("signup_form"):
        username = st.text_input("Username")
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        confirm = st.text_input("Confirm Password", type="password")
        question = st.selectbox("Security Question", questions)
        answer = st.text_input("Security Answer")

        submit = st.form_submit_button("Signup")

        if submit:
            if not all([username, email, password, confirm, answer]):
                st.error("All fields are mandatory")

            elif not valid_email(email):
                st.error("Invalid Email format")

            elif not valid_password(password):
                st.error("Password must be alphanumeric")

            elif password != confirm:
                st.error("Passwords do not match")

            else:
                try:
                    cursor.execute(
                        "INSERT INTO users VALUES (?,?,?,?,?)",
                        (username, email, password, question, answer)
                    )
                    conn.commit()

                    token = create_token({"email": email})
                    st.session_state.token = token

                    st.success("Signup successful")
                    st.session_state.page = "dashboard"
                    st.rerun()

                except:
                    st.error("Email already registered")

    if st.button("Go to Login"):
        st.session_state.page = "login"
        st.rerun()

# ---------------- LOGIN ----------------
def login():
    st.title("Login Page")

    with st.form("login_form"):
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        submit = st.form_submit_button("Login")

        if submit:
            cursor.execute(
                "SELECT username,password FROM users WHERE email=?",
                (email,)
            )
            data = cursor.fetchone()

            if data and data[1] == password:
                token = create_token({"email": email})
                st.session_state.token = token
                st.session_state.page = "dashboard"
                st.success("Login successful")
                st.rerun()
            else:
                st.error("Invalid credentials")

    if st.button("Forgot Password"):
        st.session_state.page = "forgot"
        st.rerun()

    if st.button("Create Account"):
        st.session_state.page = "signup"
        st.rerun()

# ---------------- DASHBOARD ----------------
def dashboard():
    payload = jwt.decode(
        st.session_state.token,
        SECRET_KEY,
        algorithms=[ALGORITHM]
    )

    email = payload["email"]

    cursor.execute(
        "SELECT username FROM users WHERE email=?",
        (email,)
    )
    username = cursor.fetchone()[0]

    st.title("Dashboard")
    st.success(f"Welcome {username} 🎉")

    if st.button("Logout"):
        st.session_state.token = None
        st.session_state.page = "login"
        st.rerun()

# ---------------- FORGOT PASSWORD ----------------
def forgot_password():
    st.title("Forgot Password")

    email = st.text_input("Enter Email")

    if st.button("Verify Email"):
        cursor.execute(
            "SELECT security_question FROM users WHERE email=?",
            (email,)
        )
        data = cursor.fetchone()

        if data:
            st.session_state.reset_email = email
            st.session_state.question = data[0]
        else:
            st.error("Email not found")

    if "question" in st.session_state:
        st.write(st.session_state.question)
        answer = st.text_input("Your Answer")

        if st.button("Verify Answer"):
            cursor.execute(
                "SELECT security_answer FROM users WHERE email=?",
                (st.session_state.reset_email,)
            )
            real_answer = cursor.fetchone()[0]

            if answer == real_answer:
                new_pass = st.text_input(
                    "New Password",
                    type="password"
                )

                if st.button("Update Password"):
                    if valid_password(new_pass):
                        cursor.execute(
                            "UPDATE users SET password=? WHERE email=?",
                            (new_pass, st.session_state.reset_email)
                        )
                        conn.commit()
                        st.success("Password updated")
                    else:
                        st.error("Password must be alphanumeric")
            else:
                st.error("Wrong answer")

# ---------------- ROUTING ----------------
if st.session_state.token:
    dashboard()

else:
    if st.session_state.page == "signup":
        signup()
    elif st.session_state.page == "forgot":
        forgot_password()
    else:
        login()
"""
with open("app.py", "w") as f:
    f.write(app_code)
print("Updated Streamlit app written to app.py")

Streamlit app code written to 'app.py'


In [ ]:
# --- Wait for Streamlit to Start ---
def wait_for_streamlit(port=8501, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                sock.close()
                return True
            sock.close()
        except Exception:
            pass
        time.sleep(1)
    return False
# --- Ngrok Setup ---
print("\nTo access the app, you need an Ngrok Authtoken.")
print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
authtoken = input("Enter your Ngrok Authtoken: ").strip()
if authtoken:
    ngrok.set_auth_token(authtoken)

    # Kill any existing ngrok process
    os.system("pkill ngrok")
    os.system("pkill streamlit")

    # Run Streamlit in the background FIRST
    print("Starting Streamlit...")
    # Using Subprocess.Popen to run in background
    # Redirecting output to /dev/null to keep cell clean
    process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "localhost"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Wait for it to be ready
    if wait_for_streamlit():
        print("Streamlit is active! Connecting Ngrok...")
        # Open a tunnel to the streamlit port 8501
        try:
            public_url = ngrok.connect(8501).public_url
            print(f"\n🚀 Streamlit App is running!")
            print(f"👉 Public URL: {public_url}")
            print("\n(Click the URL above to open the app)")

            # Keep main thread alive
            try:
                # Keep checking if process is alive
                while process.poll() is None:
                    time.sleep(1)
            except KeyboardInterrupt:
                print("Stopping...")
                ngrok.disconnect(public_url)
                process.terminate()
        except Exception as e:
            print(f"Ngrok connection failed: {e}")
            process.terminate()
    else:
        print("Error: Streamlit failed to start in time.")
        process.terminate()
else:
    print("Ngrok Authtoken is required to expose the app publicly.")


To access the app, you need an Ngrok Authtoken.
Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
Enter your Ngrok Authtoken: 39VeSDNVC7IJrtt4WpBEcxepb5m_7e22DsZn38MdLaWVVuYSx
Starting Streamlit...
Streamlit is active! Connecting Ngrok...

🚀 Streamlit App is running!
👉 Public URL: https://electrometrically-syntonous-jeanetta.ngrok-free.dev

(Click the URL above to open the app)


Stopping...
Ngrok connection failed: Remote end closed connection without response
